SOAL 2 :

Tunjukkan hasil performance (Confusion matrix, Accuracy, Presision, Recall, F1 Score) dan beri Analisa. Jelaskan juga tuning hyperparameter yang digunakan untuk mendapat hasil performance yang optimal . Adapun metode dan vektorisasi yang dilakukkan adalah sebagai berikut:

•	Multibert /MBERT (10)

•	LSTM dengan vektorisasi Fastext (10)

•	CNN1 dimension dengan vektorisasi Glove (10)



In [ ]:
# Bersihkan cache pip untuk menghindari konflik
!pip cache purge

# Uninstall dependensi lama
!pip uninstall -y numpy pandas tensorflow gensim scikit-learn

# Instal dependensi dengan versi spesifik
!pip install numpy==1.23.5 pandas==2.1.4 tensorflow==2.17.0 gensim==4.3.3 scikit-learn==1.5.0

# Restart runtime
import os
os.kill(os.getpid(), 9)

Files removed: 102
Found existing installation: numpy 1.24.3
Uninstalling numpy-1.24.3:
  Successfully uninstalled numpy-1.24.3
Found existing installation: pandas 2.1.4
Uninstalling pandas-2.1.4:
  Successfully uninstalled pandas-2.1.4
Found existing installation: tensorflow 2.17.0
Uninstalling tensorflow-2.17.0:
  Successfully uninstalled tensorflow-2.17.0
Found existing installation: gensim 4.3.3
Uninstalling gensim-4.3.3:
  Successfully uninstalled gensim-4.3.3
Found existing installation: scikit-learn 1.5.0
Uninstalling scikit-learn-1.5.0:
  Successfully uninstalled scikit-learn-1.5.0
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.1/17.1 MB 12.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.2/12.2 MB 15.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 601.3/601.3 MB 1.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 26.7/26.7 MB 29.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.3/13.3 MB 64.2 MB/s eta 0:00:00
ERROR: 

Soal 2.a:

Model: MultiBERT
Arsitektur:
- Pre-trained Model: BERT (bert-base-multilingual-cased), 12 layer transformer, 768 hidden size, 12 attention heads, 110M parameter
- Classification Head:
  - BERT Output: [CLS] token embedding (768 dimensi)
  - Pooler: Dense layer dengan aktivasi tanh (768 dimensi)
  - Dropout: 0.1
  - Dense (Output): 3 unit (untuk 3 kelas), aktivasi softmax (via loss function)
- Preprocessing:
  - Tokenizer: BertTokenizer (bert-base-multilingual-cased)
  - Max Sequence Length: 128
Hyperparameter:
- Optimizer: AdamW (learning rate 2e-5)
- Loss: CrossEntropyLoss
- Batch Size: 16
- Epochs: 3

In [ ]:
# MultBERT.ipynb
# Langkah 1: Import library
import pandas as pd
import torch
from transformers import BertTokenizer, BertForSequenceClassification
from torch.utils.data import DataLoader, Dataset
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, accuracy_score, precision_recall_fscore_support
from google.colab import files
import json
import numpy as np

# Periksa versi library untuk debugging
print("Versi numpy:", np.__version__)
print("Versi pandas:", pd.__version__)
print("Versi torch:", torch.__version__)
import transformers
print("Versi transformers:", transformers.__version__)
import sklearn
print("Versi scikit-learn:", sklearn.__version__)

# Langkah 2: Unggah dataset
print("Unggah file dataset: amazon_reviews_unbalanced.csv")
try:
    uploaded = files.upload()
    df = pd.read_csv('amazon_reviews_unbalanced.csv')
except Exception as e:
    print(f"Error saat mengunggah atau membaca dataset: {e}")
    raise

# Preprocessing
try:
    texts = df['text'].values
    labels = df['sentiment'].map({'Positif': 2, 'Netral': 1, 'Negatif': 0}).values
except KeyError as e:
    print(f"Error: Kolom 'text' atau 'sentiment' tidak ditemukan di dataset. {e}")
    raise

# Dataset kustom untuk BERT
class AmazonDataset(Dataset):
    def __init__(self, texts, labels, tokenizer, max_len=128):
        self.texts = texts
        self.labels = labels
        self.tokenizer = tokenizer
        self.max_len = max_len

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        text = str(self.texts[idx])
        label = self.labels[idx]
        encoding = self.tokenizer.encode_plus(
            text,
            add_special_tokens=True,
            max_length=self.max_len,
            return_token_type_ids=False,
            padding='max_length',
            truncation=True,
            return_attention_mask=True,
            return_tensors='pt'
        )
        return {
            'input_ids': encoding['input_ids'].flatten(),
            'attention_mask': encoding['attention_mask'].flatten(),
            'labels': torch.tensor(label, dtype=torch.long)
        }

# Inisialisasi tokenizer dan model
try:
    tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-cased')
    model = BertForSequenceClassification.from_pretrained('bert-base-multilingual-cased', num_labels=3)
except Exception as e:
    print(f"Error saat menginisialisasi tokenizer atau model: {e}")
    raise

# Split data
X_train, X_test, y_train, y_test = train_test_split(texts, labels, test_size=0.2, random_state=42)
train_dataset = AmazonDataset(X_train, y_train, tokenizer)
test_dataset = AmazonDataset(X_test, y_test, tokenizer)
train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=16)

# Setup pelatihan
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)
optimizer = torch.optim.AdamW(model.parameters(), lr=2e-5)

# Pelatihan
epochs = 3
for epoch in range(epochs):
    model.train()
    total_loss = 0
    for batch in train_loader:
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['labels'].to(device)
        outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss
        total_loss += loss.item()
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()
    print(f'Epoch {epoch+1}/{epochs}, Loss: {total_loss/len(train_loader):.4f}')

# Evaluasi
model.eval()
predictions, true_labels = [], []
with torch.no_grad():
    for batch in test_loader:
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['labels'].to(device)
        outputs = model(input_ids, attention_mask=attention_mask)
        preds = torch.argmax(outputs.logits, dim=1)
        predictions.extend(preds.cpu().numpy())
        true_labels.extend(labels.cpu().numpy())

# Hitung metrik
cm = confusion_matrix(true_labels, predictions)
acc = accuracy_score(true_labels, predictions)
prec, rec, f1, _ = precision_recall_fscore_support(true_labels, predictions, average='weighted')

# Simpan hasil
results = {
    'model': 'MultBERT',
    'confusion_matrix': cm.tolist(),
    'accuracy': acc,
    'precision': prec,
    'recall': rec,
    'f1_score': f1
}
with open('multbert_results.json', 'w') as f:
    json.dump(results, f)
files.download('multbert_results.json')

# Tampilkan hasil
print(f'\nMultBERT Results:')
print(f'Confusion Matrix:\n{cm}')
print(f'Accuracy: {acc:.4f}')
print(f'Precision: {prec:.4f}')
print(f'Recall: {rec:.4f}')
print(f'F1 Score: {f1:.4f}')

Versi numpy: 1.25.2
Versi pandas: 2.1.4
Versi torch: 2.3.0+cu121
Versi transformers: 4.41.2
Versi scikit-learn: 1.5.0
Unggah file dataset: amazon_reviews_unbalanced.csv


Saving amazon_reviews_unbalanced.csv to amazon_reviews_unbalanced.csv


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/996k [00:00<?, ?B/s]

...

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-multilingual-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch 1/3, Loss: 0.5238
Epoch 2/3, Loss: 0.2939
Epoch 3/3, Loss: 0.1266


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>


MultBERT Results:
Confusion Matrix:
[[ 21   0  44]
 [  0  19   0]
 [  1   0 215]]
Accuracy: 0.8500
Precision: 0.8678
Recall: 0.8500
F1 Score: 0.8197


Soal 2.b

Model: LSTM FastText


Arsitektur:
- Embedding Layer: FastText 300d (vocab_size, embedding_dim=300, input_length=100, trainable=False)
- LSTM: 64 unit, return_sequences=True
- LSTM: 32 unit
- Dropout: 0.5
- Dense: 16 unit, aktivasi ReLU
- Dense (Output): 3 unit, aktivasi softmax
Hyperparameter:
- Optimizer: Adam (default learning rate)
- Loss: Sparse Categorical Crossentropy
- Batch Size: 16
- Epochs: 5
- Validation Split: 0.2
- Max Sequence Length: 100
- Vocab Size: 5000 (num_words)

In [ ]:
# LSTM_FastText.ipynb
# Langkah 1: Import library
import pandas as pd
import numpy as np
import tensorflow as tf
# Import gensim explicitly here
import gensim
from gensim.models import KeyedVectors
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, accuracy_score, precision_recall_fscore_support
from google.colab import files
from google.colab import drive
import json
import os
import zipfile
import requests
import shutil
import gc

# Periksa versi library untuk debugging
try:
    print("Versi pandas:", pd.__version__)
    print("Versi numpy:", np.__version__)
    print("Versi tensorflow:", tf.__version__)
    # Ensure gensim is imported before checking its version
    import gensim
    print("Versi gensim:", gensim.__version__)
    import sklearn
    print("Versi scikit-learn:", sklearn.__version__)
except Exception as e:
    print(f"Error: Library tidak ditemukan. Pastikan dependensi sudah diinstal. {e}")
    raise ValueError("Library missing")

# Periksa GPU
print("GPU tersedia:", tf.config.list_physical_devices('GPU'))

# Mount Google Drive
print("Mounting Google Drive...")
drive.mount('/content/drive')

# Langkah 2: Unduh dan unggah FastText ke Google Drive
fasttext_file = 'crawl-300d-2M.vec'
fasttext_zip_file = 'crawl-300d-2M.vec.zip'
fasttext_url = 'https://dl.fbaipublicfiles.com/fasttext/vectors-english/crawl-300d-2M.vec.zip'
drive_path = '/content/drive/MyDrive/crawl-300d-2M.vec.zip'

# Hapus file lokal yang mungkin korup
for file in [fasttext_file, fasttext_zip_file]:
    if os.path.exists(file):
        print(f"Menghapus file {file} yang mungkin korup...")
        os.remove(file)

# Periksa apakah file sudah ada di Google Drive
if os.path.exists(drive_path):
    print(f"File {fasttext_zip_file} sudah ada di Google Drive. Menggunakan file tersebut...")
    shutil.copy(drive_path, fasttext_zip_file)
else:
    # Unduh FastText
    print("Mengunduh FastText...")
    try:
        response = requests.get(fasttext_url, stream=True)
        # Periksa apakah respons adalah file ZIP
        content_type = response.headers.get('Content-Type', '')
        if 'application/zip' not in content_type and 'application/octet-stream' not in content_type:
            print(f"Error: Respons bukan file ZIP (Content-Type: {content_type}).")
            print("Respons mungkin halaman HTML. Memeriksa konten...")
            content_preview = response.content[:100].decode('utf-8', errors='ignore')
            print(f"Konten awal: {content_preview}")
            raise ValueError("File yang diunduh bukan file ZIP.")

        with open(fasttext_zip_file, 'wb') as f:
            for chunk in response.iter_content(chunk_size=8192):
                if chunk:
                    f.write(chunk)
        print(f"Unduhan selesai: {fasttext_zip_file}")

        # Unggah ke Google Drive
        print(f"Mengunggah {fasttext_zip_file} ke Google Drive...")
        shutil.copy(fasttext_zip_file, drive_path)
        print(f"File berhasil diunggah ke {drive_path}")
    except Exception as e:
        print(f"Error saat mengunduh FastText: {e}")
        print("Opsi: Unggah file 'crawl-300d-2M.vec.zip' secara manual.")
        uploaded = files.upload()
        if fasttext_zip_file in uploaded:
            print(f"File {fasttext_zip_file} berhasil diunggah.")
            # Unggah ke Google Drive
            shutil.copy(fasttext_zip_file, drive_path)
            print(f"File berhasil diunggah ke {drive_path}")
        else:
            raise FileNotFoundError(f"File {fasttext_zip_file} tidak diunggah.")

# Verifikasi file ZIP
file_size = os.path.getsize(fasttext_zip_file)
if file_size < 1000000000:  # Harus ~1.2 GB
    print(f"File {fasttext_zip_file} terlalu kecil ({file_size} bytes). Kemungkinan korup.")
    raise ValueError(f"File {fasttext_zip_file} tidak valid.")

# Ekstrak file ZIP
print("Mengekstrak crawl-300d-2M.vec.zip...")
try:
    with zipfile.ZipFile(fasttext_zip_file, 'r') as zip_ref:
        zip_ref.extractall()  # Ekstrak semua file
        # Asumsi file .vec ada di root ZIP
        for file in zip_ref.namelist():
            if file.endswith('.vec'):
                os.rename(file, fasttext_file)
                break
        else:
            raise FileNotFoundError("File .vec tidak ditemukan di dalam ZIP.")
    print(f"Ekstraksi selesai: {fasttext_file}")
except Exception as e:
    print(f"Error saat mengekstrak file FastText: {e}")
    raise

# Verifikasi file hasil ekstraksi
if not os.path.exists(fasttext_file):
    raise FileNotFoundError(f"File {fasttext_file} tidak ditemukan.")
file_size = os.path.getsize(fasttext_file)
if file_size < 4000000000:  # Harus ~4 GB
    print(f"File {fasttext_file} terlalu kecil ({file_size} bytes). Kemungkinan korup.")
    raise ValueError(f"File {fasttext_file} tidak valid.")

# Langkah 3: Unggah dataset
print("Unggah file dataset: amazon_reviews_unbalanced.csv")
try:
    uploaded = files.upload()
    dataset_file = next((f for f in uploaded.keys() if 'amazon_reviews_unbalanced' in f), None)
    if not dataset_file:
        raise FileNotFoundError("File dataset tidak ditemukan. Pastikan nama file mengandung 'amazon_reviews_unbalanced'.")
    df = pd.read_csv(dataset_file)
except Exception as e:
    print(f"Error saat mengunggah atau membaca dataset: {e}")
    raise

# Preprocessing
try:
    texts = df['text'].values
    labels = df['sentiment'].map({'Positif': 2, 'Netral': 1, 'Negatif': 0}).values
except KeyError as e:
    print(f"Error: Kolom 'text' atau 'sentiment' tidak ditemukan di dataset. {e}")
    raise

# Tokenisasi
tokenizer = Tokenizer(num_words=5000)
tokenizer.fit_on_texts(texts)
sequences = tokenizer.texts_to_sequences(texts)
X = pad_sequences(sequences, maxlen=100)
gc.collect()  # Bersihkan memori

# Split data
X_train, X_test, y_train, y_test = train_test_split(X, labels, test_size=0.2, random_state=42)

# Muat model FastText
try:
    print("Memuat model FastText...")
    fasttext_model = KeyedVectors.load_word2vec_format(fasttext_file, binary=False, encoding='utf-8', unicode_errors='replace')
    print("Model FastText berhasil dimuat.")
except Exception as e:
    print(f"Error saat memuat model FastText: {e}")
    raise

vocab_size = len(tokenizer.word_index) + 1
embedding_dim = 300
embedding_matrix = np.zeros((vocab_size, embedding_dim))
for word, i in tokenizer.word_index.items():
    try:
        embedding_matrix[i] = fasttext_model[word]
    except KeyError:
        continue
del fasttext_model  # Hapus model dari memori
gc.collect()

# Bangun model LSTM
model = Sequential([
    Embedding(vocab_size, embedding_dim, weights=[embedding_matrix], input_length=100, trainable=False),
    LSTM(64, return_sequences=True),  # Kurangi unit untuk menghemat memori
    LSTM(32),
    Dropout(0.5),
    Dense(16, activation='relu'),
    Dense(3, activation='softmax')
])

# Kompilasi model
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Latih model
try:
    model.fit(X_train, y_train, epochs=5, batch_size=16, validation_split=0.2, verbose=1)
except Exception as e:
    print(f"Error saat melatih model: {e}")
    raise

# Evaluasi
y_pred = np.argmax(model.predict(X_test), axis=1)
cm = confusion_matrix(y_test, y_pred)
acc = accuracy_score(y_test, y_pred)
prec, rec, f1, _ = precision_recall_fscore_support(y_test, y_pred, average='weighted')

# Simpan hasil
results = {
    'model': 'LSTM FastText',
    'confusion_matrix': cm.tolist(),
    'accuracy': acc,
    'precision': prec,
    'recall': rec,
    'f1_score': f1
}
with open('lstm_fasttext_results.json', 'w') as f:
    json.dump(results, f)
files.download('lstm_fasttext_results.json')

# Tampilkan hasil
print(f'\nLSTM FastText Results:')
print(f'Confusion Matrix:\n{cm}')
print(f'Accuracy: {acc:.4f}')
print(f'Precision: {prec:.4f}')
print(f'Recall: {rec:.4f}')
print(f'F1 Score: {f1:.4f}')

Versi pandas: 2.1.4
Versi numpy: 1.26.4
Versi tensorflow: 2.17.0
Versi gensim: 4.3.3
Versi scikit-learn: 1.5.0
GPU tersedia: []
Mounting Google Drive...
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Mengunduh FastText...
Unduhan selesai: crawl-300d-2M.vec.zip
Mengunggah crawl-300d-2M.vec.zip ke Google Drive...
File berhasil diunggah ke /content/drive/MyDrive/crawl-300d-2M.vec.zip
Mengekstrak crawl-300d-2M.vec.zip...
Ekstraksi selesai: crawl-300d-2M.vec
Unggah file dataset: amazon_reviews_unbalanced.csv


Saving amazon_reviews_unbalanced.csv to amazon_reviews_unbalanced (2).csv
Memuat model FastText...
Model FastText berhasil dimuat.


/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/embedding.py:97: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Epoch 1/5
60/60 ━━━━━━━━━━━━━━━━━━━━ 27s 143ms/step - accuracy: 0.6709 - loss: 0.8907 - val_accuracy: 0.6500 - val_loss: 0.8207
Epoch 2/5
60/60 ━━━━━━━━━━━━━━━━━━━━ 8s 113ms/step - accuracy: 0.7212 - loss: 0.6855 - val_accuracy: 0.7042 - val_loss: 0.7007
Epoch 3/5
60/60 ━━━━━━━━━━━━━━━━━━━━ 12s 134ms/step - accuracy: 0.7714 - loss: 0.5815 - val_accuracy: 0.8083 - val_loss: 0.5266
Epoch 4/5
60/60 ━━━━━━━━━━━━━━━━━━━━ 6s 108ms/step - accuracy: 0.8499 - loss: 0.3972 - val_accuracy: 0.7792 - val_loss: 0.4843
Epoch 5/5
60/60 ━━━━━━━━━━━━━━━━━━━━ 10s 111ms/step - accuracy: 0.8284 - loss: 0.4271 - val_accuracy: 0.8042 - val_loss: 0.6558
10/10 ━━━━━━━━━━━━━━━━━━━━ 1s 109ms/step


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>


LSTM FastText Results:
Confusion Matrix:
[[ 32   1  32]
 [  0  11   8]
 [  3   0 213]]
Accuracy: 0.8533
Precision: 0.8623
Recall: 0.8533
F1 Score: 0.8376


In [6]:
# Tambahkan path lingkungan virtual
import sys
sys.path.append('/content/venv/lib/python3.11/site-packages')

# Verifikasi versi library
try:
    import numpy
    import tensorflow
    import pandas
    import sklearn
    import jax
    print("Versi numpy:", numpy.__version__)  # Target: 1.25.2
    print("Versi tensorflow:", tensorflow.__version__)  # Target: 2.17.0
    print("Versi pandas:", pandas.__version__)  # Target: 2.2.2
    print("Versi scikit-learn:", sklearn.__version__)  # Target: 1.5.0
    print("Versi jax:", jax.__version__)  # Target: 0.4.23
except ImportError as e:
    print(f"Error: Library tidak ditemukan. Menginstal ulang dependensi. {e}")
    !pip cache purge
    !pip install virtualenv
    !virtualenv /content/venv
    !source /content/venv/bin/activate
    !/content/venv/bin/pip install --upgrade pip
    !/content/venv/bin/pip uninstall -y numpy pandas tensorflow scikit-learn jax jaxlib
    !/content/venv/bin/pip install --force-reinstall numpy==1.25.2 pandas==2.2.2 tensorflow==2.17.0 scikit-learn==1.5.0 jax==0.4.23 jaxlib==0.4.23
    import numpy
    import tensorflow
    import pandas
    import sklearn
    import jax
    print("Versi numpy:", numpy.__version__)
    print("Versi tensorflow:", tensorflow.__version__)
    print("Versi pandas:", pandas.__version__)
    print("Versi scikit-learn:", sklearn.__version__)
    print("Versi jax:", jax.__version__)

Versi numpy: 2.0.2
Versi tensorflow: 2.18.0
Versi pandas: 2.2.2
Versi scikit-learn: 1.6.1
Versi jax: 0.5.2


Soal 2.c

Model: CNN GloVe


Arsitektur:
- Embedding Layer: GloVe 300d (vocab_size, embedding_dim=300, input_length=100, trainable=False)
- Conv1D: 128 filter, kernel size 5, aktivasi ReLU
- GlobalMaxPooling1D
- Dropout: 0.5
- Dense: 64 unit, aktivasi ReLU
- Dropout: 0.5
- Dense (Output): 3 unit, aktivasi softmax
Hyperparameter:
- Optimizer: Adam (default learning rate)
- Loss: Sparse Categorical Crossentropy
- Batch Size: 16
- Epochs: 5
- Validation Split: 0.2
- Max Sequence Length: 100
- Vocab Size: 5000 (num_words)

In [8]:
# CNN_GloVe.ipynb
# Langkah 1: Import library
import sys
sys.path.append('/content/venv/lib/python3.11/site-packages')  # Pastikan lingkungan virtual digunakan
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Conv1D, GlobalMaxPooling1D, Dense, Dropout
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, accuracy_score, precision_recall_fscore_support
from google.colab import drive
from google.colab import files
import json
import os
import zipfile
import requests
import shutil
import gc
import hashlib

# Periksa versi library untuk debugging
try:
    print("Versi pandas:", pd.__version__)
    print("Versi numpy:", np.__version__)
    print("Versi tensorflow:", tf.__version__)
    print("Versi scikit-learn:", sklearn.__version__)
except Exception as e:
    print(f"Error: Library tidak ditemukan. Pastikan dependensi sudah diinstal. {e}")
    raise ValueError("Library missing")

# Periksa GPU
print("GPU tersedia:", tf.config.list_physical_devices('GPU'))

# Mount Google Drive
print("Mounting Google Drive...")
drive.mount('/content/drive', force_remount=True)

# Langkah 2: Unduh dan unggah GloVe ke Google Drive
glove_file = 'glove.6B.300d.txt'
glove_zip_file = 'glove.6B.300d.txt.zip'
glove_url = 'http://nlp.stanford.edu/data/glove.6B.zip'
drive_path = '/content/drive/MyDrive/glove.6B.300d.txt.zip'

# Hapus file lokal yang mungkin korup
for file in [glove_file, glove_zip_file]:
    if os.path.exists(file):
        print(f"Menghapus file {file} yang mungkin korup...")
        os.remove(file)

# Periksa apakah file ada di Google Drive
if os.path.exists(drive_path):
    file_size = os.path.getsize(drive_path)
    if file_size < 800000000:  # Harus ~822 MB
        print(f"File {drive_path} di Google Drive terlalu kecil ({file_size} bytes). Menghapus...")
        os.remove(drive_path)
    else:
        # Verifikasi integritas file ZIP dengan checksum (opsional)
        print(f"Memeriksa integritas {drive_path}...")
        with open(drive_path, 'rb') as f:
            file_hash = hashlib.sha256(f.read()).hexdigest()
        print(f"SHA256: {file_hash}")
        # Salin file dari Google Drive ke lokal
        print(f"Menyalin {drive_path} ke {glove_zip_file}...")
        shutil.copy(drive_path, glove_zip_file)
        # Verifikasi file lokal ada setelah disalin
        if not os.path.exists(glove_zip_file):
            raise FileNotFoundError(f"Gagal menyalin {drive_path} ke {glove_zip_file}. Pastikan ada cukup ruang di Colab.")
        print(f"File berhasil disalin ke {glove_zip_file}")

# Unduh GloVe jika tidak ada di Google Drive
if not os.path.exists(drive_path):
    print("Mengunduh GloVe...")
    try:
        response = requests.get(glove_url, stream=True)
        # Periksa apakah respons adalah file ZIP
        content_type = response.headers.get('Content-Type', '')
        if 'application/zip' not in content_type and 'application/octet-stream' not in content_type:
            print(f"Error: Respons bukan file ZIP (Content-Type: {content_type}).")
            print("Respons mungkin halaman HTML. Memeriksa konten...")
            content_preview = response.content[:100].decode('utf-8', errors='ignore')
            print(f"Konten awal: {content_preview}")
            raise ValueError("File yang diunduh bukan file ZIP.")

        with open(glove_zip_file, 'wb') as f:
            total_size = 0
            for chunk in response.iter_content(chunk_size=8192):
                if chunk:
                    f.write(chunk)
                    total_size += len(chunk)
                    print(f"Unduh: {total_size / 1024 / 1024:.2f} MB", end='\r')
        print(f"\nUnduhan selesai: {glove_zip_file}")

        # Verifikasi ukuran file ZIP
        file_size = os.path.getsize(glove_zip_file)
        if file_size < 800000000:  # Harus ~822 MB
            print(f"File {glove_zip_file} terlalu kecil ({file_size} bytes). Kemungkinan korup.")
            raise ValueError(f"File {glove_zip_file} tidak valid.")

        # Unggah ke Google Drive
        print(f"Mengunggah {glove_zip_file} ke Google Drive...")
        shutil.copy(glove_zip_file, drive_path)
        print(f"File berhasil diunggah ke {drive_path}")
    except Exception as e:
        print(f"Error saat mengunduh GloVe: {e}")
        raise FileNotFoundError(f"Gagal mengunduh {glove_zip_file}. Unggah file secara manual ke {drive_path} dan jalankan kembali.")

# Ekstrak file ZIP
print("Mengekstrak glove.6B.300d.txt.zip...")
try:
    with zipfile.ZipFile(glove_zip_file, 'r') as zip_ref:
        zip_ref.extractall()  # Ekstrak semua file
        # Cari file glove.6B.300d.txt
        for file in zip_ref.namelist():
            if file == 'glove.6B.300d.txt':
                os.rename(file, glove_file)
                break
        else:
            raise FileNotFoundError("File glove.6B.300d.txt tidak ditemukan di dalam ZIP.")
    print(f"Ekstraksi selesai: {glove_file}")
except Exception as e:
    print(f"Error saat mengekstrak file GloVe: {e}")
    raise

# Verifikasi keberadaan file (tanpa pengecekan ukuran)
if not os.path.exists(glove_file):
    raise FileNotFoundError(f"File {glove_file} tidak ditemukan.")
file_size = os.path.getsize(glove_file)
print(f"Ukuran file {glove_file}: {file_size / 1024 / 1024 / 1024:.2f} GB")

# Langkah 3: Baca dataset dari Google Drive
dataset_path = '/content/drive/MyDrive/colab/amazon_reviews_unbalanced.csv'
print(f"Membaca dataset dari {dataset_path}...")
try:
    if not os.path.exists(dataset_path):
        raise FileNotFoundError(f"File dataset tidak ditemukan di {dataset_path}. Pastikan file ada di folder Colab.")
    df = pd.read_csv(dataset_path)
    print("Dataset berhasil dimuat.")
except Exception as e:
    print(f"Error saat membaca dataset: {e}")
    raise

# Preprocessing
try:
    texts = df['text'].values
    labels = df['sentiment'].map({'Positif': 2, 'Netral': 1, 'Negatif': 0}).values
except KeyError as e:
    print(f"Error: Kolom 'text' atau 'sentiment' tidak ditemukan di dataset. {e}")
    raise

# Tokenisasi
tokenizer = Tokenizer(num_words=5000)
tokenizer.fit_on_texts(texts)
sequences = tokenizer.texts_to_sequences(texts)
X = pad_sequences(sequences, maxlen=100)
gc.collect()  # Bersihkan memori

# Split data
X_train, X_test, y_train, y_test = train_test_split(X, labels, test_size=0.2, random_state=42)

# Muat embedding GloVe
print("Memuat embedding GloVe...")
embeddings_index = {}
try:
    with open(glove_file, encoding='utf-8') as f:
        for line in f:
            values = line.split()
            word = values[0]
            coefs = np.asarray(values[1:], dtype='float32')
            embeddings_index[word] = coefs
    print("Embedding GloVe berhasil dimuat.")
except Exception as e:
    print(f"Error saat memuat embedding GloVe: {e}")
    raise

# Buat embedding matrix
vocab_size = len(tokenizer.word_index) + 1
embedding_dim = 300
embedding_matrix = np.zeros((vocab_size, embedding_dim))
for word, i in tokenizer.word_index.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector
del embeddings_index  # Hapus dari memori
gc.collect()

# Bangun model 1D CNN
model = Sequential([
    Embedding(vocab_size, embedding_dim, weights=[embedding_matrix], input_length=100, trainable=False),
    Conv1D(128, 5, activation='relu'),
    GlobalMaxPooling1D(),
    Dropout(0.5),
    Dense(64, activation='relu'),
    Dropout(0.5),
    Dense(3, activation='softmax')
])

# Kompilasi model
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Latih model
try:
    model.fit(X_train, y_train, epochs=5, batch_size=16, validation_split=0.2, verbose=1)
except Exception as e:
    print(f"Error saat melatih model: {e}")
    raise

# Evaluasi
y_pred = np.argmax(model.predict(X_test), axis=1)
cm = confusion_matrix(y_test, y_pred)
acc = accuracy_score(y_test, y_pred)
prec, rec, f1, _ = precision_recall_fscore_support(y_test, y_pred, average='weighted')

# Simpan hasil
results = {
    'model': 'CNN GloVe',
    'confusion_matrix': cm.tolist(),
    'accuracy': acc,
    'precision': prec,
    'recall': rec,
    'f1_score': f1
}
with open('cnn_glove_results.json', 'w') as f:
    json.dump(results, f)
files.download('cnn_glove_results.json')

# Tampilkan hasil
print(f'\nCNN GloVe Results:')
print(f'Confusion Matrix:\n{cm}')
print(f'Accuracy: {acc:.4f}')
print(f'Precision: {prec:.4f}')
print(f'Recall: {rec:.4f}')
print(f'F1 Score: {f1:.4f}')

Versi pandas: 2.2.2
Versi numpy: 2.0.2
Versi tensorflow: 2.18.0
Versi scikit-learn: 1.6.1
GPU tersedia: [PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]
Mounting Google Drive...
Mounted at /content/drive
Menghapus file glove.6B.300d.txt yang mungkin korup...
Menghapus file glove.6B.300d.txt.zip yang mungkin korup...
Memeriksa integritas /content/drive/MyDrive/glove.6B.300d.txt.zip...
SHA256: 617afb2fe6cbd085c235baf7a465b96f4112bd7f7ccb2b2cbd649fed9cbcf2fb
Menyalin /content/drive/MyDrive/glove.6B.300d.txt.zip ke glove.6B.300d.txt.zip...
File berhasil disalin ke glove.6B.300d.txt.zip
Mengekstrak glove.6B.300d.txt.zip...
Ekstraksi selesai: glove.6B.300d.txt
Ukuran file glove.6B.300d.txt: 0.97 GB
Membaca dataset dari /content/drive/MyDrive/colab/amazon_reviews_unbalanced.csv...
Dataset berhasil dimuat.
Memuat embedding GloVe...
Embedding GloVe berhasil dimuat.


/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Epoch 1/5
60/60 ━━━━━━━━━━━━━━━━━━━━ 6s 17ms/step - accuracy: 0.5705 - loss: 1.0586 - val_accuracy: 0.6500 - val_loss: 0.8024
Epoch 2/5
60/60 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.6848 - loss: 0.7894 - val_accuracy: 0.6500 - val_loss: 0.7715
Epoch 3/5
60/60 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.7031 - loss: 0.7042 - val_accuracy: 0.6500 - val_loss: 0.7050
Epoch 4/5
60/60 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.7429 - loss: 0.5426 - val_accuracy: 0.7000 - val_loss: 0.6493
Epoch 5/5
60/60 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.7947 - loss: 0.5227 - val_accuracy: 0.6750 - val_loss: 0.6610
10/10 ━━━━━━━━━━━━━━━━━━━━ 1s 46ms/step


/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>


CNN GloVe Results:
Confusion Matrix:
[[  7   0  58]
 [  3   0  16]
 [  3   0 213]]
Accuracy: 0.7333
Precision: 0.6510
Recall: 0.7333
F1 Score: 0.6487


In [ ]:
# Sel 1: Instalasi ulang dependensi dan restart runtime
import os

# Hapus versi lama numpy dan pandas
print("Menghapus versi lama numpy dan pandas...")
!pip uninstall -y numpy pandas

# Instal ulang numpy dan pandas dengan versi yang kompatibel
print("Menginstal numpy==2.0.2 dan pandas==2.2.2...")
!pip install numpy==2.0.2 pandas==2.2.2

# Restart runtime agar perubahan diterapkan
print("Merestart runtime...")
os.kill(os.getpid(), 9)  # Ini akan merestart runtime Colab

Menghapus versi lama numpy dan pandas...
Found existing installation: numpy 2.0.2
Uninstalling numpy-2.0.2:
  Successfully uninstalled numpy-2.0.2
Found existing installation: pandas 2.0.3
Uninstalling pandas-2.0.3:
  Successfully uninstalled pandas-2.0.3
Menginstal numpy==2.0.2 dan pandas==2.2.2...
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.9/60.9 kB 4.2 MB/s eta 0:00:00
  Using cached pandas-2.2.2-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (19 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.5/19.5 MB 70.7 MB/s eta 0:00:00
Using cached pandas-2.2.2-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (13.0 MB)


In [5]:
# Compare_Models.ipynb
import json
import pandas as pd
import numpy as np
from google.colab import drive
import os

# Verifikasi versi library untuk debugging
print("Versi pandas:", pd.__version__)  # Harus 2.2.2
print("Versi numpy:", np.__version__)  # Harus 2.0.2

# Mount Google Drive
print("Mounting Google Drive...")
drive.mount('/content/drive', force_remount=True)

# Path ke file JSON
multibert_path = '/content/drive/MyDrive/colab/multbert_results.json'
cnn_glove_path = '/content/drive/MyDrive/colab/cnn_glove_results.json'
lstm_fasttext_path = '/content/drive/MyDrive/colab/lstm_fasttext_results.json'

# Periksa keberadaan file
for path in [multibert_path, cnn_glove_path, lstm_fasttext_path]:
    if not os.path.exists(path):
        raise FileNotFoundError(f"File {path} tidak ditemukan.")

# Baca file JSON
with open(multibert_path, 'r') as f:
    multibert_results = json.load(f)
with open(cnn_glove_path, 'r') as f:
    cnn_glove_results = json.load(f)
with open(lstm_fasttext_path, 'r') as f:
    lstm_fasttext_results = json.load(f)

# Tampilkan arsitektur model
print("\n=== Arsitektur Model ===")

# Arsitektur MultiBERT (berdasarkan MultBERT.ipynb)
print("\nModel: MultiBERT")
print("Arsitektur:")
print("- Pre-trained Model: BERT (bert-base-multilingual-cased), 12 layer transformer, 768 hidden size, 12 attention heads, 110M parameter")
print("- Classification Head:")
print("  - BERT Output: [CLS] token embedding (768 dimensi)")
print("  - Pooler: Dense layer dengan aktivasi tanh (768 dimensi)")
print("  - Dropout: 0.1")
print("  - Dense (Output): 3 unit (untuk 3 kelas), aktivasi softmax (via loss function)")
print("- Preprocessing:")
print("  - Tokenizer: BertTokenizer (bert-base-multilingual-cased)")
print("  - Max Sequence Length: 128")
print("Hyperparameter:")
print("- Optimizer: AdamW (learning rate 2e-5)")
print("- Loss: CrossEntropyLoss")
print("- Batch Size: 16")
print("- Epochs: 3")

# Arsitektur CNN GloVe (berdasarkan CNN_GloVe.ipynb)
print("\nModel: CNN GloVe")
print("Arsitektur:")
print("- Embedding Layer: GloVe 300d (vocab_size, embedding_dim=300, input_length=100, trainable=False)")
print("- Conv1D: 128 filter, kernel size 5, aktivasi ReLU")
print("- GlobalMaxPooling1D")
print("- Dropout: 0.5")
print("- Dense: 64 unit, aktivasi ReLU")
print("- Dropout: 0.5")
print("- Dense (Output): 3 unit, aktivasi softmax")
print("Hyperparameter:")
print("- Optimizer: Adam (default learning rate)")
print("- Loss: Sparse Categorical Crossentropy")
print("- Batch Size: 16")
print("- Epochs: 5")
print("- Validation Split: 0.2")
print("- Max Sequence Length: 100")
print("- Vocab Size: 5000 (num_words)")

# Arsitektur LSTM FastText (berdasarkan LSTM_FastText.ipynb)
print("\nModel: LSTM FastText")
print("Arsitektur:")
print("- Embedding Layer: FastText 300d (vocab_size, embedding_dim=300, input_length=100, trainable=False)")
print("- LSTM: 64 unit, return_sequences=True")
print("- LSTM: 32 unit")
print("- Dropout: 0.5")
print("- Dense: 16 unit, aktivasi ReLU")
print("- Dense (Output): 3 unit, aktivasi softmax")
print("Hyperparameter:")
print("- Optimizer: Adam (default learning rate)")
print("- Loss: Sparse Categorical Crossentropy")
print("- Batch Size: 16")
print("- Epochs: 5")
print("- Validation Split: 0.2")
print("- Max Sequence Length: 100")
print("- Vocab Size: 5000 (num_words)")

# Ekstrak metrik dari ketiga model
models = [
    multibert_results['model'],
    cnn_glove_results['model'],
    lstm_fasttext_results['model']
]
accuracy = [
    multibert_results['accuracy'],
    cnn_glove_results['accuracy'],
    lstm_fasttext_results['accuracy']
]
precision = [
    multibert_results['precision'],
    cnn_glove_results['precision'],
    lstm_fasttext_results['precision']
]
recall = [
    multibert_results['recall'],
    cnn_glove_results['recall'],
    lstm_fasttext_results['recall']
]
f1_score = [
    multibert_results['f1_score'],
    cnn_glove_results['f1_score'],
    lstm_fasttext_results['f1_score']
]

# Buat DataFrame untuk tabel perbandingan
comparison_df = pd.DataFrame({
    'Model': models,
    'Accuracy': accuracy,
    'Precision': precision,
    'Recall': recall,
    'F1-Score': f1_score
})

# Format angka ke 4 desimal
comparison_df['Accuracy'] = comparison_df['Accuracy'].round(4)
comparison_df['Precision'] = comparison_df['Precision'].round(4)
comparison_df['Recall'] = comparison_df['Recall'].round(4)
comparison_df['F1-Score'] = comparison_df['F1-Score'].round(4)

# Tampilkan tabel perbandingan
print("\n=== Tabel Perbandingan Model ===")
display(comparison_df)

# Simpan tabel ke CSV
comparison_df.to_csv('/content/drive/MyDrive/colab/model_comparison.csv', index=False)
print("Tabel perbandingan disimpan ke /content/drive/MyDrive/colab/model_comparison.csv")

# Tampilkan matriks konfusi untuk masing-masing model
print("\n=== Matriks Konfusi ===")

print("\nMatriks Konfusi MultiBERT:")
print(pd.DataFrame(multibert_results['confusion_matrix'],
                   index=['Negatif', 'Netral', 'Positif'],
                   columns=['Prediksi Negatif', 'Prediksi Netral', 'Prediksi Positif']))

print("\nMatriks Konfusi CNN GloVe:")
print(pd.DataFrame(cnn_glove_results['confusion_matrix'],
                   index=['Negatif', 'Netral', 'Positif'],
                   columns=['Prediksi Negatif', 'Prediksi Netral', 'Prediksi Positif']))

print("\nMatriks Konfusi LSTM FastText:")
print(pd.DataFrame(lstm_fasttext_results['confusion_matrix'],
                   index=['Negatif', 'Netral', 'Positif'],
                   columns=['Prediksi Negatif', 'Prediksi Netral', 'Prediksi Positif']))

Versi pandas: 2.2.2
Versi numpy: 2.0.2
Mounting Google Drive...
Mounted at /content/drive

=== Arsitektur Model ===

Model: MultiBERT
Arsitektur:
- Pre-trained Model: BERT (bert-base-multilingual-cased), 12 layer transformer, 768 hidden size, 12 attention heads, 110M parameter
- Classification Head:
  - BERT Output: [CLS] token embedding (768 dimensi)
  - Pooler: Dense layer dengan aktivasi tanh (768 dimensi)
  - Dropout: 0.1
  - Dense (Output): 3 unit (untuk 3 kelas), aktivasi softmax (via loss function)
- Preprocessing:
  - Tokenizer: BertTokenizer (bert-base-multilingual-cased)
  - Max Sequence Length: 128
Hyperparameter:
- Optimizer: AdamW (learning rate 2e-5)
- Loss: CrossEntropyLoss
- Batch Size: 16
- Epochs: 3

Model: CNN GloVe
Arsitektur:
- Embedding Layer: GloVe 300d (vocab_size, embedding_dim=300, input_length=100, trainable=False)
- Conv1D: 128 filter, kernel size 5, aktivasi ReLU
- GlobalMaxPooling1D
- Dropout: 0.5
- Dense: 64 unit, aktivasi ReLU
- Dropout: 0.5
- Dense (Out

,Model,Accuracy,Precision,Recall,F1-Score
0,MultBERT,0.8500,0.8678,0.8500,0.8197
1,CNN GloVe,0.7333,0.6510,0.7333,0.6487
2,LSTM FastText,0.8533,0.8623,0.8533,0.8376


Tabel perbandingan disimpan ke /content/drive/MyDrive/colab/model_comparison.csv

=== Matriks Konfusi ===

Matriks Konfusi MultiBERT:
         Prediksi Negatif  Prediksi Netral  Prediksi Positif
Negatif                21                0                44
Netral                  0               19                 0
Positif                 1                0               215

Matriks Konfusi CNN GloVe:
         Prediksi Negatif  Prediksi Netral  Prediksi Positif
Negatif                 7                0                58
Netral                  3                0                16
Positif                 3                0               213

Matriks Konfusi LSTM FastText:
         Prediksi Negatif  Prediksi Netral  Prediksi Positif
Negatif                32                1                32
Netral                  0               11                 8
Positif                 3                0               213
